In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
from sklearn.model_selection import train_test_split

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [2]:
RANDOM_SEED = 17

In [3]:
dataset = pd.read_csv("data/train_data.csv",index_col=False)
# dataset_test = pd.read_csv("data/test_data.csv",index_col=False)

In [4]:
dataset_train, dataset_test = train_test_split(dataset,test_size = 0.2,random_state = RANDOM_SEED)

In [5]:
dataset_train.head

<bound method NDFrame.head of        index                            title  topic_idx
25339  25339         더민주 서영교 여파 지역위원장 심사기준 강화          6
24704  24704      맛집에 너그러운 한국인 해외여행서도 JMT 찾았다          3
1834    1834        특징주 삼성물산 지배구조 이슈 부각에 강세종합          1
17604  17604   생필품난 베네수엘라 콜롬비아와의 국경 1년 만에 재개방          4
19362  19362        금태섭 국민 10명 중 8명 판결문 공개 원해          6
...      ...                              ...        ...
25631  25631      7번째 개관하는 소극장 운동 발원지 삼일로창고극장          3
42297  42297    삼강엠앤티 516억원 규모 케미컬 탱크 3척 공급계약          1
33174  33174    이란 외무장관 美 제재 해제하면 협상의 문 활짝 열려          4
34959  34959     제주·남부 지방에 호우특보…완도 165.5㎜ 장대비          3
10863  10863  영천 새마을금고 강도 범행 6시간만에 검거…범행동기 조사          2

[36523 rows x 3 columns]>

In [20]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset[sent_key]]
        
        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"
            
        if self.mode == "train":
            self.labels = [np.int32(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int32(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            return (self.sentences[i] + (self.labels[i], ))
        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))


using cached model


In [9]:

## Setting parameters
max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
NUM_CLASS = 7

data_train = BERTDataset(dataset_train, "title", "topic_idx", tok, max_len, True, False)
data_test = BERTDataset(dataset_test, "title", "topic_idx", tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_CLASS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [10]:

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1142 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9620752334594727 train acc 0.21875
epoch 1 batch id 201 loss 0.5817193984985352 train acc 0.6344838308457711
epoch 1 batch id 401 loss 0.4634286165237427 train acc 0.7600529925187033
epoch 1 batch id 601 loss 0.3617643415927887 train acc 0.8068843594009983
epoch 1 batch id 801 loss 0.26881682872772217 train acc 0.8291588639200999
epoch 1 batch id 1001 loss 0.3131040036678314 train acc 0.8432192807192808
epoch 1 train acc 0.8510562609457093


/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/286 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8802447552447552


  0%|          | 0/1142 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1405194103717804 train acc 0.9375
epoch 5 batch id 201 loss 0.029889114201068878 train acc 0.970771144278607
epoch 5 batch id 401 loss 0.06615283340215683 train acc 0.9748285536159601
epoch 5 batch id 601 loss 0.009841816499829292 train acc 0.9757175540765392
epoch 5 batch id 801 loss 0.007665166165679693 train acc 0.9776451310861424
epoch 5 batch id 1001 loss 0.10059695690870285 train acc 0.9792395104895105
epoch 5 train acc 0.9802429947460596


  0%|          | 0/286 [00:00<?, ?it/s]

epoch 5 test acc 0.8798076923076923


In [21]:
dataset_test = pd.read_csv("data/test_data.csv",index_col=False)
data_test = BERTDataset(dataset_test, "title", None, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

submission = pd.read_csv('data/sample_submission.csv')

In [24]:
model.eval()
pred = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    for o in out:
        pred.append(o)
    
submission['topic_idx'] = pred

/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/286 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.92 GiB total capacity; 10.10 GiB already allocated; 6.06 MiB free; 10.17 GiB reserved in total by PyTorch)

In [25]:
torch.save(model.state_dict(),"models/kobert.pth")